In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from pathlib import Path
import pandas as pd
import xarray as xr
import h5py
from astropy.time import Time

pd.set_option('max_colwidth', 400)

%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

## Load some of our own functions
* A `setup.py` is added to have easier access to our functions
* Under `overcast` root directory, run `pip install -e .` . 
* ***RESTART*** the kernel of this notebook, then you are good to go.


In [3]:
from overcast import readers as rd
from overcast.utils import fn2rgt, datainfo, query_rgt, box_sel

## Archive all the data we have a region
* Find all the files for each product
* put the RGT, cycle_number, and filename into Pandas DataFrame so we can match granules. This is done by calling `datainfo` in `utils.py`.

In [4]:
ROI = 'BEAUFORT' ## ['LAPTEV','CHUKCHI']
dataroot = Path.home()/'shared/data-overcast'/ROI

fns07 = sorted( (dataroot/'ATL07').glob('*.h5') )
fns09 = sorted( (dataroot/'ATL09').glob('*.h5') )
db07 = datainfo(fns07)
db09 = datainfo(fns09)
db07.head()

,rgt,cycle_number,filename
0,1112,3,/home/jovyan/shared/data-overcast/BEAUFORT/ATL07/processed_ATL07-01_20190610013526_11120301_003_02.h5
1,1119,3,/home/jovyan/shared/data-overcast/BEAUFORT/ATL07/processed_ATL07-01_20190610123527_11190301_003_02.h5
2,1127,3,/home/jovyan/shared/data-overcast/BEAUFORT/ATL07/processed_ATL07-01_20190611010946_11270301_003_02.h5
3,1134,3,/home/jovyan/shared/data-overcast/BEAUFORT/ATL07/processed_ATL07-01_20190611120948_11340301_003_02.h5
4,1142,3,/home/jovyan/shared/data-overcast/BEAUFORT/ATL07/processed_ATL07-01_20190612004407_11420301_003_02.h5


## Select a granule and find the files from ATL07 and ATL09 database
* This uses a simple function `query_rgt` in `utils.py`

In [ ]:
rgt = 1142; cyc = 3

fn07 = query_rgt(rgt,cyc,db07)[0]
fn09 = query_rgt(rgt,cyc,db09)[0]
print(fn07)
print(fn09)

## Check space orientation. 
* ATL09 is strong beam only so we have to find the matching ATL07 beam
* Choose the center beam for example

In [ ]:
f07 = h5py.File(fn07, 'r')
#check to see if it is forward (1)  or backward (0) orientation to know which beam is strong/weak
print(f07['orbit_info/sc_orient'][0])
f07.close()

In [ ]:
beam07 = 'gt2l'
beam09 = 'profile_2'

## Read ATL07 and ATL09 data
* For ATL07, the reader from the sea ice tutorial is used.
* A `getATL09` reader is added to `readers.py` but using xarray because we want to include some 2D data.
* Another advantage of xarray dataset is that now we can add attributes to variables.

In [ ]:
with h5py.File(fn07, 'r') as f07:
    epoch=f07['/ancillary_data/atlas_sdp_gps_epoch'][0]
    df07= rd.getATL07(f07,beam07)
    df07['time']=Time(epoch+df07['dt'],format='gps').utc.datetime
df07.head()

In [ ]:
with h5py.File(fn09, 'r') as f09:
    epoch=f09['/ancillary_data/atlas_sdp_gps_epoch'][0]
    ds09= rd.getATL09(f09,beam09)
    ds09['time']=(['dt'],Time(epoch+ds09['dt'],format='gps').utc.datetime)
ds09

## Select a region we want to plot
* Here we used the same bounding box as we used yesterday
* both ATL07 and ATL09 data are selected by calling the same function `box_sel` in `utils.py`. 
* Check `ds09_cut` and `df07_cut` to make sure they are in our box

In [ ]:
bbox = [-150,75,-110,78]
df07_cut=df07[box_sel(df07,bbox)]
ds09_cut=ds09.where(box_sel(ds09,bbox),drop=True)

In [ ]:
ds09_cut

In [ ]:
df07_cut

## Get the clout top and bottom height from ATL09
* This is a simple case, where only one layer of cloud is present
* At this time of year, no blowing snow either.

In [ ]:
laytop = ds09_cut['layer_top'].values
laytop[laytop>20e3] = -100
ctop = laytop.max(axis=1)
laybot = ds09_cut['layer_bot'].values
cbot = laybot.min(axis=1)

## Plot some ERA5 fields with the granule maps
* find the closest time from the hourly ERA5 data for convenience. 

In [ ]:
erafn = Path.home()/'shared/data-overcast/ERA5/BEAUFORT_201906.nc'
dsera = xr.open_dataset(erafn)
dsera = dsera.rename({'lat':'lats','lon':'lons'})

In [ ]:
tm07 = df07_cut['time'].mean()
tm09 = ds09_cut['time'].mean().values


In [ ]:
#dsera_cut = dsera.where( box_sel(dsera,bbox,to_360=True) , drop=True)
#dsera_cut = dsera_cut.sel(time0=tm07.round('H'))
dsera_cut = dsera.sel(time0=tm07.round('H'))
dsera_cut = dsera_cut.sel(time1=tm07.round('H'))
dsera_cut

## First plot in a map to see where the granule is 

In [ ]:
bea_proj = ccrs.NorthPolarStereo(central_longitude=-140)
pcproj   = ccrs.PlateCarree()

var = 'heights'
vmin = 0; vmax= 2
ticks=np.arange(vmin,vmax,.2)
fig = plt.figure(1,figsize=(8,8),dpi=100)
ax1 = fig.add_subplot(211,projection=bea_proj)
ax1.coastlines(resolution='50m', color='black', linewidth=1)
ax1.set_extent([bbox[0],bbox[2],bbox[1],bbox[3]], pcproj)
ax1.gridlines(draw_labels=True)
cs = ax1.contour(dsera_cut.lons,dsera_cut.lats,dsera_cut.slp/100,colors='k',transform=pcproj)
ax1.clabel(cs,cs.levels, inline=True,fontsize=10,fmt='%.1f')
hdl = plt.scatter(df07_cut['lons'][::100], df07_cut['lats'][::100],c=df07_cut[var][::100], 
            cmap=plt.cm.get_cmap('RdYlGn').reversed(), 
            vmin=vmin,vmax=vmax,
            transform=pcproj)
plt.colorbar(label=var, shrink=1, ticks=ticks,extend='both');

ax2 = fig.add_subplot(212,projection=bea_proj)
ax2.coastlines(resolution='50m', color='black', linewidth=1)
ax2.set_extent([bbox[0],bbox[2],bbox[1],bbox[3]], ccrs.PlateCarree())
ax2.gridlines(draw_labels=True)
plt.scatter(ds09_cut['lons'][::20], ds09_cut['lats'][::20],c=ctop[::20], 
            cmap=plt.cm.get_cmap('gist_ncar_r').reversed(), 
            vmin=100,vmax=1000,
            transform=pcproj)
plt.colorbar(label='cloud top height \n [m]', shrink=1, ticks=np.arange(100,1000,50),extend='both');

## Plot ATL07 and ATL09 data along the same latitude coordinate
* It seem the laser is totally attenuated near the right end of the plots so very little surface and column optical depth retrieval
* so it is likely cloud on the left and on the right are of different origin?

In [ ]:
fig,ax=plt.subplots(6,1,figsize=(15,24),sharex=True,constrained_layout=True)

i = 0
ax[i].scatter(df07_cut.lats,df07_cut.photon_rate,s=2,c='gray', label='ATL07 all surface type')
ax[i].scatter(df07_cut[df07_cut.stype>1].lats,df07_cut[df07_cut.stype>1].photon_rate,s=10,c='b', label= 'ATL07 surface type lead')
ax[i].legend(fontsize=15)
ax[i].set_ylim(0,20)
ax[i].set_ylabel('ATL07 \nphoton rate',fontsize=15)

i+=1
ax[i].scatter(df07_cut.lats,df07_cut.gauss,s=2,c='gray', label='ATL07 all surface type')
ax[i].scatter(df07_cut[df07_cut.stype>1].lats,df07_cut[df07_cut.stype>1].gauss,s=10,c='blue',label= 'ATL07 surface type lead')
ax[i].legend(loc=1,fontsize=15)
ax[i].set_ylabel('ATL07 \ngaussian width',fontsize=15)
ax[i].set_ylim(0,.6)

i+=1
ax[i].grid()
ax[i].scatter(df07_cut.lats,df07_cut.stype,s=2,c='gray',label='ATL07 all surface type')
ax[i].scatter(df07_cut[df07_cut.stype>1].lats,df07_cut[df07_cut.stype>1].stype,s=10,c='blue',label= 'ATL07 surface type lead')
ax[i].legend(fontsize=15)
ax[i].set_ylim(-1,10)
ax[i].set_ylabel('ATL07 \nsurface type',fontsize=15)
ax[i].set_yticks(np.arange(0,10,1))

i+=1
ax[i].scatter(df07_cut.lats,df07_cut.ssh_flag,c='k',s=2,marker='.',label='ATL07 all surface type')
ax[i].scatter(df07_cut[df07_cut.stype>1].lats,df07_cut[df07_cut.stype>1].ssh_flag,c='b',s=10,label='ATL07 surface type lead')
ax[i].legend(fontsize=15)
ax[i].grid()
ax[i].set_ylim(-1,2)
ax[i].set_yticks(np.arange(0,2,1))
ax[i].set_yticklabels(['sea ice', 'sea surface'],fontsize=12)
ax[i].set_ylabel('ATL07 ssh flag',fontsize=15)

i+=1
hdl = ax[i].pcolormesh(ds09_cut['lats'],ds09_cut['zbin'],ds09_cut['cab_prof'].T,
                      vmin=0,vmax=1e-3,
                      cmap=plt.cm.gist_ncar)
ax[i].set_ylim([0,1000])
cb = fig.colorbar(hdl, ax=ax[i],extend='both',aspect=10)
ax[i].plot(ds09_cut['lats'],ctop,'r')
ax[i].plot(ds09_cut['lats'],cbot,'r')
ax[i].set_ylabel('ATL09 backscatting coefficient',fontsize=16)

i+=1
ax[i].scatter(ds09_cut.lats,ds09_cut.column_od_asr,c='k',s=10,marker='.')
ax[i].set_ylabel('ATL09 column optical depth',fontsize=16)
ax[i].set_ylim([0,5])
ax[i].grid()


ax[i].set_xlabel('latitude',fontsize=15)

In [ ]:
## Failed attempt to use widget 
# bea_proj = ccrs.NorthPolarStereo(central_longitude=-140)
# pcproj   = ccrs.PlateCarree()

# var = 'heights'
# vmin = 0; vmax= 2
# ticks=np.arange(vmin,vmax,.2)
# fig = plt.figure(1,figsize=(8,8),dpi=100)
# ax1 = fig.add_subplot(211,projection=bea_proj)
# ax1.coastlines(resolution='50m', color='black', linewidth=1)
# ax1.set_extent([bbox[0],bbox[2],bbox[1],bbox[3]], pcproj)
# ax1.gridlines(draw_labels=True)
# plt.scatter(df07_cut['lons'][::100], df07_cut['lats'][::100],c=df07_cut[var][::100], 
#             cmap=plt.cm.get_cmap('RdYlGn').reversed(), 
#             vmin=vmin,vmax=vmax,
#             transform=pcproj)
# plt.colorbar(label=var, shrink=0.5, ticks=ticks,extend='both');

# laytop = ds09_cut['layer_top'].values
# laytop[laytop>20e3] = -100
# ctop = laytop.max(axis=1)
# ax2 = fig.add_subplot(212,sharex=ax1,sharey=ax1,projection=bea_proj)
# ax2.coastlines(resolution='50m', color='black', linewidth=1)
# ax2.set_extent([bbox[0],bbox[2],bbox[1],bbox[3]], ccrs.PlateCarree())
# ax2.gridlines(draw_labels=True)
# plt.scatter(ds09_cut['lons'][::20], ds09_cut['lats'][::20],c=ctop[::20], 
#             cmap=plt.cm.get_cmap('gist_ncar_r').reversed(), 
#             vmin=100,vmax=1000,
#             transform=pcproj)
# plt.colorbar(label='cloud top height', shrink=0.5, ticks=np.arange(100,1000,50),extend='both');